## Creating and solving your Application 1 with Gurobi

### Restaurant Capacity Maximization Problem



**Decision Variables**

$f_i$ : 1 if table i  is used, 0 otherwise, where $i=1...N$

$g_{ij}:$ 1 if both table i and j are used, 0 otherwise,  

$x_i =$ bottom left x coordinate of table i

$y_i =$ bottom left x coordinate of table i

$l_{ij} =$ 1 if table i is located left to j

$b_{ij} =$ 1 if table i is located below j


**Parameters**

$d$ : The minimum physical distance required between tables

$c_i =$ max # people that can be seated @ table i, where $i=1...N$

$C$ = Minimum desired establishment capacity

$w_i =$ width of table i

$h_i =$ height of table i

$R_w =$ width of dining space

$R_h =$ height of dining space

$U_{ij}$ = Underperformance measure of staff when table i and j are used


**Objective Function**

$min$ $z$ = $g_1^+ + g_2^- +\sum_{i=1}^{N}\sum_{j=1, i\neq j}^{N}g_{ij}U_{ij}$


**Constraints**

**s.t**
Force non-linear element to be 0 or 1:

$g_{ij} >= f_i+f_j-1 \ i, j \in T and \ i \neq j$

No overlap if the following constraint is true:

$l_{ij} + l_{ji} + b_{ij} + b_{ji} + (1-x_i) + (1-x_j) >= 1, \ i, j \in T, \ i<j$ 

If a table i is to the left of table j, table i's right edge x coordinate  must be less than table j's x left edge x coordinate:

$x_i +w_i <= x_j + W - Wl_{ij}, \ i, j \in T$

If table i is below j, table i's top edge y coordinate must be less than table j's bottom edge y coordinate:

$y_i +h_i <= y_j + H - Hb_{ij}, \ i, j \in T$

Tables must exist within the dining space:

$x_i >= 0, \ i \in T$

$x_i+w_i <= R_x, \ i \in T$

$y_i >= 0, \ i \in T$

$y_i+h_i <= R_h, \ i \in T$

Unused table space must be under U:

$R_wR_h \ - \sum_{i=1}^{N}f_i(w_i)(h_i) - (g_1^+-g_1^-) = U, \ i \in T$ 

Facility capacity must be over C to remain profitable

$\sum_{i=1}^{N}c_if_i - (g_2^+-g_2^-) = C, \ i \in T$

Binary variable constraints:

$l_{ij}, b_{ij} \ \in {0,1}$

$f_{i} \ \in {0,1}$

$g_{ij}\in {0,1}$

#### Step 1: Import grobipy module

In [237]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
from gurobipy import quicksum
import plotly.graph_objects as go

#### Step 2: Define the model

In [238]:
m = gp.Model()

#### Step 3: Import generated instance data

In [239]:
# this step is done in the programmatic looping solution

#### Step 4: Define the parameters

In [240]:

    #number of tables
    N = 3
    
    #Minimum physical distane required between tables
    d = 2
    
    #capacity for each table
    c = [2, 2, 2]
    
    #width for each table
    w = [1, 1, 2]
   
    #account for physical distancing margin
    for i in range(len(w)):
        w[i] += d

    #height for each table
    h = [1, 2, 2]
    
    #account for physical distancing margin
    for i in range(len(h)):
        h[i] += d
    
    #Width of dining space
    Rw = 7

    #Height of dining space
    Rh = 7
    
    #Unused table space goal
    U = 2

    #Minimum desired establishment capacity goal
    C = 10

    #Underperformance measure of staff when table i and j are used Uij[rows][columns]
    Uij =  [[1,1,1],
            [1,1,1],
            [1,1,1]]

#### Step 5: Define decision variables

In [241]:
#binary, if table i is used
f = {}
for i in range(N):
    f[i] = m.addVar(vtype=GRB.BINARY, name="f_" +str(i))

#bottom left x coordinate of table i
x = {}
for i in range(N):
    x[i] = m.addVar(vtype=GRB.CONTINUOUS, name="x_"+str(i))
    
#bottom left y coordinate of table i
y = {}
for i in range(N):
    y[i] = m.addVar(vtype=GRB.CONTINUOUS, name="y_"+str(i))
    
    
#binary, if table i is located left to table j
l = {}
for i in range(N):
    for j in range(N):
        if i != j:
            l[i, j] = m.addVar(vtype=GRB.BINARY, name="l_" +str(i)+str(j))

#binary, if table i is located below to table j
b = {}
for i in range(N):
    for j in range(N):
        if i != j:
            b[i, j] = m.addVar(vtype=GRB.BINARY, name="b_" +str(i)+str(j))

#binary, if table i and table j are both used
g = {}
for i in range(N):
    for j in range(N):
        if i != j:
            g[i, j] = m.addVar(vtype=GRB.BINARY, name="g_" +str(i)+str(j))

#surplus for goal 1  
g1surplus={}
g1surplus = m.addVar(vtype=GRB.CONTINUOUS, name="g1surplus")
    
#slack for goal 1  
g1slack={}
g1slack = m.addVar(vtype=GRB.CONTINUOUS, name="g1slack")

#surplus for goal 2
g2surplus={}
g2surplus= m.addVar(vtype=GRB.CONTINUOUS, name="g2surplus")

#slack/surplus for goal 2
g2slack={}
g2slack= m.addVar(vtype=GRB.CONTINUOUS, name="g2slack")

#### Step 6: Set the objective function

In [242]:
nonlinearobj = quicksum((g[i, j]*Uij[i][j])/2 for i in range(N) for j in range(N) if i != j)
m.setObjective(g1surplus+g2slack+(nonlinearobj), GRB.MINIMIZE)

#### Step 7: Add constraints

In [243]:
# no overlap
m.addConstrs(l[i,j] + l[j,i] + b[i,j] + b[j,i] + (1-f[i]) + (1-f[j])  >= 1 for i in range(N) for j in range(N) if i != j)

# If a table i is to the left of table j, table i's right edge x coordinate  must be less than table j's x left edge x coordinate
m.addConstrs(x[i] + w[i] <= (x[j] + Rw - Rw*l[i,j]) for i in range(N) for j in range(N) if i != j)

#If table i is below j, table i's top edge y coordinate must be less than table j's bottom edge y coordinate
m.addConstrs(y[i] + h[i] <= (y[j] + Rh - Rh*b[i,j]) for i in range(N) for j in range(N) if i != j)

#table space must exist within the dining space
m.addConstrs(x[i] >= 0 for i in range(N))
m.addConstrs(x[i]+w[i]<= Rw for i in range (N))
m.addConstrs(y[i] >= 0 for i in range(N))
m.addConstrs(y[i]+h[i]<= Rh for i in range (N))

#Unused table space must be under U - GOAL 1
m.addConstrs((Rw*Rh) - quicksum((w[i])*(h[i])*f[i] for i in range(N)) - g1surplus + g1slack == U for i in range(N))

#Facility capacity must be over C to remain profitable - GOAL 2
m.addConstrs(quicksum(f[i]*c[i] for i in range(N)) + g2slack - g2surplus == C for i in range(N))

#Quadratic constraint
m.addConstrs(g[i,j] >= f[i] + f[j] - 1 for i in range(N) for j in range(N) if i != j)

#update model
m.update()

#### Step 8: Solve the model

In [244]:
m.optimize()

Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 42 rows, 31 columns and 132 nonzeros
Model fingerprint: 0xb5d3c80a
Variable types: 10 continuous, 21 integer (21 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [5e-01, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+01]
Found heuristic solution: objective 39.0000000
Presolve removed 26 rows and 11 columns
Presolve time: 0.00s
Presolved: 16 rows, 20 columns, 53 nonzeros
Variable types: 3 continuous, 17 integer (17 binary)

Root relaxation: objective 1.700000e+01, 6 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   17.00000    0    6   39.00000   17.00000  56.4%     -    0s
H    0     0                      29.0000000   17.00000  41.4%     -    0s
H    0     0                      17.0000000   17.00000  0.00%    

#### Insert variable values

In [245]:
#Parameters
print("Room size:", Rw*Rh, "m^2")
#Tables:
for i in f:
    print("Table", i, "was", "not" if i==0 else "", "used.", "Position:", "(", x[i].x,",",y[i].x,").", "Capacity:", c[i], "Table space height:", h[i], "Table space width:", w[i])
#Goal 1 print
print("Goal 1: Unused space must be under", U, "m^2.", )
print("Goal 1 Result: Unused space was", g1surplus.x, "m^2.", )
#Goal 2 print
print("Goal 2: Facility capacity must be over", C, "to remain profitable.", )
print("Goal 2 Result: Facility capacity was unmet by", g2slack.x, ".", )
#nonlinear element print
print("Underperformance level for this table setup is", nonlinearobj.getValue())
tableUsedData = []
for tableUsed in f:
    tableUsedData.append(f[tableUsed].x)
xCoords = []
for i in x:
    xCoords.append(x[i].x)
yCoords = []
for i in y:
    yCoords.append(y[i].x)
d = {'ifUsed' : tableUsedData, 'x-coord' : xCoords, 'y-coord' : yCoords, 'capacity' : c, 'table space width' : w, 'table space height' : h}
df = pd.DataFrame(data=d)
df



Room size: 49 m^2
Table 0 was not used. Position: ( 0.0 , 0.0 ). Capacity: 2 Table space height: 3 Table space width: 3
Table 1 was  used. Position: ( 4.0 , 0.0 ). Capacity: 2 Table space height: 4 Table space width: 3
Table 2 was  used. Position: ( 0.0 , 3.0 ). Capacity: 2 Table space height: 4 Table space width: 4
Goal 1: Unused space must be under 2 m^2.
Goal 1 Result: Unused space was 10.0 m^2.
Goal 2: Facility capacity must be over 10 to remain profitable.
Goal 2 Result: Facility capacity was unmet by 4.0 .
Underperformance level for this table setup is 3.0


,ifUsed,x-coord,y-coord,capacity,table space width,table space height
0,1.0,0.0,0.0,2,3,3
1,1.0,4.0,0.0,2,3,4
2,1.0,0.0,3.0,2,4,4


In [246]:
xCoordPlot = []
yCoordPlot = []
for i in f:
    if(f[i].x == 1):
        #top border
        xCoordPlot.append(0)
        xCoordPlot.append(Rw)
        yCoordPlot.append(Rh)
        yCoordPlot.append(Rh)
        xCoordPlot.append("none")
        yCoordPlot.append("none")
        
        #right border
        xCoordPlot.append(Rw)
        xCoordPlot.append(Rw)
        yCoordPlot.append(0)
        yCoordPlot.append(Rh)
        xCoordPlot.append("none")
        yCoordPlot.append("none")
        #x
        xCoordPlot.append(x[i].x)
        xCoordPlot.append(x[i].x)
        xCoordPlot.append(x[i].x + w[i])
        xCoordPlot.append(x[i].x + w[i])
        xCoordPlot.append(x[i].x)
        xCoordPlot.append("none")
        #y
        yCoordPlot.append(y[i].x)
        yCoordPlot.append(y[i].x + h[i])
        yCoordPlot.append(y[i].x + h[i])
        yCoordPlot.append(y[i].x)
        yCoordPlot.append(y[i].x)
        yCoordPlot.append("none")
fig = go.Figure(go.Scatter(x=xCoordPlot, y=yCoordPlot, fill="toself"))
fig.show()

## Programmatic looping solution

Here, we will be importing our previously generated instance data. This data exists in JSON format, and is easily loaded in a useable dict format by using the python json module. We will be solving all of these instances, and the solution data including the generated model, solution data, and decision variable values will be outputted to the '/solved_models/' directory.

### Read in generated instances

In [247]:
import json 
with open('generated_instances.json') as f:
  loaded_instances = json.load(f)

### Define a function to wrap the entire solving flow from above section

In [248]:
# instance is an object containing generated instance parameters
def solveModel(instance, iteration):
    print("INSTANCE #", iteration)
    # STEP 1: DEFINE THE MODEL
    m = gp.Model()
    
    # STEP 2: DEFINE THE PARAMETERS THAT WILL BE USED
    
    #number of tables
    N = instance['num_tables']
    print("Number of tables:", N)
    
    #Minimum physical distance required between tables
    d = 2
    
    #capacity for each table
    c = [table['capacity'] for table in instance['tables']]
    
    #width for each table
    w = [table['width'] for table in instance['tables']]
    print("Table widths:", w)
   
    #account for physical distancing margin
    for i in range(len(w)):
        w[i] += d

    #height for each table
    h = [table['height'] for table in instance['tables']]
    print("Table heights:", h)
    
    #account for physical distancing margin
    for i in range(len(h)):
        h[i] += d
    
    #Width of dining space
    Rw = instance['width']

    #Height of dining space
    Rh = instance['height']
    
    #Unused table space goal
    U = 20

    #Minimum desired establishment capacity goal
    C = 100

    #Underperformance measure of staff when table i and j are used Uij[rows][columns]
    Uij =  [[1 for table in instance['tables']] for table in instance['tables'] ]
    print("Table underperformance matrix:", Uij)
  
    # STEP 3: DEFINE DECISION VARIABLES
    
    #binary, if table i is used
    f = {}
    for i in range(N):
        f[i] = m.addVar(vtype=GRB.BINARY, name="f_" +str(i))

    #bottom left x coordinate of table i
    x = {}
    for i in range(N):
        x[i] = m.addVar(vtype=GRB.CONTINUOUS, name="x_"+str(i))

    #bottom left y coordinate of table i
    y = {}
    for i in range(N):
        y[i] = m.addVar(vtype=GRB.CONTINUOUS, name="y_"+str(i))


    #binary, if table i is located left to table j
    l = {}
    for i in range(N):
        for j in range(N):
            if i != j:
                l[i, j] = m.addVar(vtype=GRB.BINARY, name="l_" +str(i)+str(j))

    #binary, if table i is located below to table j
    b = {}
    for i in range(N):
        for j in range(N):
            if i != j:
                b[i, j] = m.addVar(vtype=GRB.BINARY, name="b_" +str(i)+str(j))

    #binary, if table i and table j are both used
    g = {}
    for i in range(N):
        for j in range(N):
            if i != j:
                g[i, j] = m.addVar(vtype=GRB.BINARY, name="g_" +str(i)+str(j))

    #surplus for goal 1  
    g1surplus={}
    g1surplus = m.addVar(vtype=GRB.CONTINUOUS, name="g1surplus")
    
    #slack for goal 1  
    g1slack={}
    g1slack = m.addVar(vtype=GRB.CONTINUOUS, name="g1slack")
    
    
    #surplus for goal 2
    g2surplus={}
    g2surplus= m.addVar(vtype=GRB.CONTINUOUS, name="g2surplus")

    #slack/surplus for goal 2
    g2slack={}
    g2slack= m.addVar(vtype=GRB.CONTINUOUS, name="g2slack")
    
    
    # STEP 4: DEFINE OBJECTIVE FUNCTION
    nonlinearobj = quicksum((g[i, j]*Uij[i][j])/2 for i in range(N) for j in range(N) if i != j)
    m.setObjective(g1surplus+g2slack+(nonlinearobj), GRB.MINIMIZE)
    
    # Step 5: ADD CONSTRAINTS
    # no overlap
    m.addConstrs(l[i,j] + l[j,i] + b[i,j] + b[j,i] + (1-f[i]) + (1-f[j])  >= 1 for i in range(N) for j in range(N) if i != j)

    # If a table i is to the left of table j, table i's right edge x coordinate  must be less than table j's x left edge x coordinate
    m.addConstrs(x[i] + w[i] <= (x[j] + Rw - Rw*l[i,j]) for i in range(N) for j in range(N) if i != j)

    #If table i is below j, table i's top edge y coordinate must be less than table j's bottom edge y coordinate
    m.addConstrs(y[i] + h[i] <= (y[j] + Rh - Rh*b[i,j]) for i in range(N) for j in range(N) if i != j)

    #table space must exist within the dining space
    m.addConstrs(x[i] >= 0 for i in range(N))
    m.addConstrs(x[i]+w[i]<= Rw for i in range (N))
    m.addConstrs(y[i] >= 0 for i in range(N))
    m.addConstrs(y[i]+h[i]<= Rh for i in range (N))

    #Unused table space must be under U - GOAL 1
    m.addConstrs((Rw*Rh) - quicksum((w[i])*(h[i])*f[i] for i in range(N)) - g1surplus + g1slack == U for i in range(N))

    #Facility capacity must be over C to remain profitable - GOAL 2
    m.addConstrs(quicksum(f[i]*c[i] for i in range(N)) + g2slack -g2surplus == C for i in range(N))

    #Quadratic constraint
    m.addConstrs(g[i,j] >= f[i] + f[j] - 1 for i in range(N) for j in range(N) if i != j)

    #update model
    m.update()
    
    # STEP 6: SOLVE THE MODEL
    m.optimize()
    
    # STEP 7: VISUALLY DISPLAY RESULTS
    #Parameters
    print("Room size:", Rw*Rh, "m^2")
    #Tables:
    for i in f:
        print("Table", i, "was", "not" if i==0 else "", "used.", "Position:", "(", x[i].x,",",y[i].x,").", "Capacity:", c[i], "Table space height:", h[i], "Table space width:", w[i])
    #Goal 1 print
    print("Goal 1: Unused space must be under", U, "m^2.", )
    print("Goal 1 Result: Unused space was", g1surplus.x, "m^2.", )
    
    #Goal 2 print
    print("Goal 2: Facility capacity must be over", C, "to remain profitable.", )
    print("Goal 2 Result: Facility capacity was unmet by", g2slack.x, ".", )
    
    #nonlinear element print
    print("Underperformance level for this table setup is", nonlinearobj.getValue())
    tableUsedData = []
    
    for tableUsed in f:
        tableUsedData.append(f[tableUsed].x)
    xCoords = []
    for i in x:
        xCoords.append(x[i].x)
    yCoords = []
    for i in y:
        yCoords.append(y[i].x)
    d = {'ifUsed' : tableUsedData, 'x-coord' : xCoords, 'y-coord' : yCoords, 'capacity' : c, 'table space width' : w, 'table space height' : h}
    df = pd.DataFrame(data=d)
    
    # STEP 8: CREATE VISUALIZATION USING PLOTLY
    xCoordPlot = []
    yCoordPlot = []
    for i in f:
        if(f[i].x == 1):
            #top border
            xCoordPlot.append(0)
            xCoordPlot.append(Rw)
            yCoordPlot.append(Rh)
            yCoordPlot.append(Rh)
            xCoordPlot.append("none")
            yCoordPlot.append("none")

            #right border
            xCoordPlot.append(Rw)
            xCoordPlot.append(Rw)
            yCoordPlot.append(0)
            yCoordPlot.append(Rh)
            xCoordPlot.append("none")
            yCoordPlot.append("none")
            #x
            xCoordPlot.append(x[i].x)
            xCoordPlot.append(x[i].x)
            xCoordPlot.append(x[i].x + w[i])
            xCoordPlot.append(x[i].x + w[i])
            xCoordPlot.append(x[i].x)
            xCoordPlot.append("none")
            #y
            yCoordPlot.append(y[i].x)
            yCoordPlot.append(y[i].x + h[i])
            yCoordPlot.append(y[i].x + h[i])
            yCoordPlot.append(y[i].x)
            yCoordPlot.append(y[i].x)
            yCoordPlot.append("none")
            
    fig = go.Figure(go.Scatter(x=xCoordPlot, y=yCoordPlot, fill="toself"))
    fig.show()
    #STEP 9: SAVE MODEL, SOLUTION, PLOTLY FIGURE, AND DECISION VARIABLE VALUES TO /solved_models/ DIRECTORY
    m.write('solved_models/instance_%d_out.lp' % iteration)
    m.write('solved_models/instance_%d_out.json' % iteration)
    #fig.write_image('solved_models/instance_%d_fig.png' % iteration, engine='kaleido')
    df.to_csv('solved_models/instance_%d_out.csv' % iteration, index=False)
    

### Solve all instances

In [249]:
iteration = 0
for instance in loaded_instances:
    solveModel(instance, iteration)
    iteration+=1

INSTANCE # 0
Number of tables: 5
Table widths: [4, 3, 3, 2, 2]
Table heights: [3, 3, 4, 2, 2]
Table underperformance matrix: [[1, 1, 1, 1, 1], [1, 1, 1, 1, 1], [1, 1, 1, 1, 1], [1, 1, 1, 1, 1], [1, 1, 1, 1, 1]]
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 110 rows, 79 columns and 390 nonzeros
Model fingerprint: 0x9f839a2d
Variable types: 14 continuous, 65 integer (65 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+01]
  Objective range  [5e-01, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Found heuristic solution: objective 275.0000000
Presolve removed 50 rows and 14 columns
Presolve time: 0.00s
Presolved: 60 rows, 65 columns, 210 nonzeros
Variable types: 10 continuous, 55 integer (55 binary)

Root relaxation: objective 1.120000e+02, 19 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time



INSTANCE # 1
Number of tables: 8
Table widths: [2, 3, 5, 2, 3, 3, 2, 2]
Table heights: [2, 3, 4, 2, 3, 3, 2, 2]
Table underperformance matrix: [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1]]
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 272 rows, 196 columns and 1032 nonzeros
Model fingerprint: 0xb2ad00d1
Variable types: 20 continuous, 176 integer (176 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+01]
  Objective range  [5e-01, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 9e+02]
Found heuristic solution: objective 902.0000000
Presolve removed 104 rows and 32 columns
Presolve time: 0.00s
Presolved: 168 rows, 164 columns, 588 nonzeros
Variable types: 16 continuous, 148 integer (148 binary)

Root relaxation: objective 7.230000e+02, 52 iterations, 0.00 seconds

 

INSTANCE # 2
Number of tables: 18
Table widths: [3, 2, 3, 3, 5, 2, 3, 2, 3, 3, 5, 3, 3, 4, 2, 2, 3, 3]
Table heights: [4, 2, 4, 3, 3, 2, 3, 2, 5, 3, 3, 3, 3, 5, 2, 2, 3, 5]
Table underperformance matrix: [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1

INSTANCE # 3
Number of tables: 5
Table widths: [3, 3, 3, 5, 3]
Table heights: [3, 3, 3, 4, 3]
Table underperformance matrix: [[1, 1, 1, 1, 1], [1, 1, 1, 1, 1], [1, 1, 1, 1, 1], [1, 1, 1, 1, 1], [1, 1, 1, 1, 1]]
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 110 rows, 79 columns and 390 nonzeros
Model fingerprint: 0x9e36bb8c
Variable types: 14 continuous, 65 integer (65 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+01]
  Objective range  [5e-01, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
Found heuristic solution: objective 224.0000000
Presolve removed 49 rows and 13 columns
Presolve time: 0.00s
Presolved: 61 rows, 66 columns, 216 nonzeros
Variable types: 10 continuous, 56 integer (56 binary)

Root relaxation: objective 4.200000e+01, 20 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time



INSTANCE # 4
Number of tables: 12
Table widths: [3, 2, 3, 2, 2, 5, 3, 3, 3, 2, 3, 3]
Table heights: [4, 2, 3, 2, 2, 3, 3, 4, 3, 2, 3, 3]
Table underperformance matrix: [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 600 rows, 436 columns and 2364 nonzeros
Model fingerprint: 0x06c2b470
Variable types: 28 continuous, 408 integer (408 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+01]
  Objective range  [5e-01, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 6e+02]
Found heuris

...

INSTANCE # 10
Number of tables: 11
Table widths: [3, 2, 3, 3, 5, 2, 3, 2, 2, 3, 3]
Table heights: [3, 2, 3, 3, 5, 2, 5, 2, 2, 3, 3]
Table underperformance matrix: [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 506 rows, 367 columns and 1980 nonzeros
Model fingerprint: 0x2ce8f4a1
Variable types: 26 continuous, 341 integer (341 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+01]
  Objective range  [5e-01, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+02]
Found heuristic solution: objective 849.0000000
Presolve removed 175 rows and 58 columns

INSTANCE # 11
Number of tables: 16
Table widths: [3, 3, 3, 2, 2, 5, 2, 2, 3, 3, 3, 2, 2, 3, 5, 2]
Table heights: [3, 3, 3, 2, 2, 5, 2, 2, 3, 3, 3, 2, 2, 5, 3, 2]
Table underperformance matrix: [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]
Gurobi

INSTANCE # 12
Number of tables: 6
Table widths: [3, 3, 3, 2, 4, 3]
Table heights: [3, 3, 3, 2, 4, 4]
Table underperformance matrix: [[1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 156 rows, 112 columns and 570 nonzeros
Model fingerprint: 0x6e54c209
Variable types: 16 continuous, 96 integer (96 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+01]
  Objective range  [5e-01, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Found heuristic solution: objective 313.0000000
Presolve removed 66 rows and 19 columns
Presolve time: 0.00s
Presolved: 90 rows, 93 columns, 315 nonzeros
Variable types: 12 continuous, 81 integer (81 binary)

Root relaxation: objective 1.360000e+02, 28 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf |

INSTANCE # 13
Number of tables: 12
Table widths: [3, 5, 2, 3, 2, 3, 4, 5, 4, 5, 2, 4]
Table heights: [4, 5, 2, 3, 2, 3, 5, 3, 3, 4, 2, 5]
Table underperformance matrix: [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 600 rows, 436 columns and 2364 nonzeros
Model fingerprint: 0xe0e00ea0
Variable types: 28 continuous, 408 integer (408 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+01]
  Objective range  [5e-01, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 9e+02]
Found heuri

INSTANCE # 14
Number of tables: 18
Table widths: [2, 3, 3, 3, 3, 2, 4, 4, 3, 3, 5, 4, 2, 2, 4, 3, 5, 3]
Table heights: [2, 3, 3, 5, 3, 2, 5, 3, 3, 4, 4, 5, 2, 2, 3, 3, 3, 3]
Table underperformance matrix: [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 

## Solutions to models also available in '/solved_models/' directory